In [1]:
import torch
import jovian
import torchvision
import tarfile
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
from torch.utils.data import random_split
from torch.utils.data import DataLoader, TensorDataset, random_split
import numpy as np
import os
from torchvision.datasets.utils import download_url
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.transforms import ToTensor
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
import pandas as pd
from torchvision.utils import make_grid
from torchvision.utils import make_grid
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

<IPython.core.display.Javascript object>

In [2]:
data=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
test_ids=test["PassengerId"]

In [64]:
test

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,1,34.5,0,0,7.8292,1
1,3,0,47.0,1,0,7.0000,2
2,2,1,62.0,0,0,9.6875,1
3,3,1,27.0,0,0,8.6625,2
4,3,0,22.0,1,1,12.2875,2
...,...,...,...,...,...,...,...
413,3,1,27.0,0,0,8.0500,2
414,1,0,39.0,0,0,108.9000,0
415,3,1,38.5,0,0,7.2500,2
416,3,1,27.0,0,0,8.0500,2


In [3]:
def clean(data):
    data=data.drop(["Name","Ticket","Cabin","PassengerId"],axis=1)#sacrificing Cabin data
    cols=["SibSp","Parch","Fare","Age"]
    for col in cols:
        data[col].fillna(data[col].median(),inplace=True)
    data.Embarked.fillna("U",inplace=True)
    return data

In [4]:
data=clean(data)
test=clean(test)

In [5]:
data.head(25)


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
5,0,3,male,28.0,0,0,8.4583,Q
6,0,1,male,54.0,0,0,51.8625,S
7,0,3,male,2.0,3,1,21.0750,S
8,1,3,female,27.0,0,2,11.1333,S
9,1,2,female,14.0,1,0,30.0708,C


In [6]:
from sklearn import preprocessing
le=preprocessing.LabelEncoder()

cols=["Sex","Embarked"]

for col in cols:
    data[col]=le.fit_transform(data[col])
    test[col]=le.transform(test[col])
    print(le.classes_)


data.head(63)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
58,1,2,0,5.0,1,2,27.7500,2
59,0,3,1,11.0,5,2,46.9000,2
60,0,3,1,22.0,0,0,7.2292,0
61,1,1,0,38.0,0,0,80.0000,3


In [7]:
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2
...,...,...,...,...,...,...,...,...
886,0,2,1,27.0,0,0,13.0000,2
887,1,1,0,19.0,0,0,30.0000,2
888,0,3,0,28.0,1,2,23.4500,2
889,1,1,1,26.0,0,0,30.0000,0


In [8]:
inputs=data.drop("Embarked",axis=1).values
labels=data[["Embarked"]].values

In [9]:
inputs

array([[ 0.    ,  3.    ,  1.    , ...,  1.    ,  0.    ,  7.25  ],
       [ 1.    ,  1.    ,  0.    , ...,  1.    ,  0.    , 71.2833],
       [ 1.    ,  3.    ,  0.    , ...,  0.    ,  0.    ,  7.925 ],
       ...,
       [ 0.    ,  3.    ,  0.    , ...,  1.    ,  2.    , 23.45  ],
       [ 1.    ,  1.    ,  1.    , ...,  0.    ,  0.    , 30.    ],
       [ 0.    ,  3.    ,  1.    , ...,  0.    ,  0.    ,  7.75  ]])

In [10]:
labels

array([[2],
       [0],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
       [2],
       [2],
       [0],
       [2],
       [2],
       [1],
       [2],
       [2],
       [2],
       [0],
       [2],
       [1],
       [2],
       [0],
       [0],
       [1],
       [2],
       [0],
       [2],
       [0],
       [2],
       [2],
       [0],
       [2],
       [2],
       [0],
       [0],
       [1],
       [2],
       [1],
       [1],
       [0],
       [2],
       [2],
       [2],
       [0],
       [2],
       [0],
       [2],
       [2],
       [0],
       [2],
       [2],
       [0],
       [3],
       [2],
       [2],
       [0],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [0],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [2],
       [1],
    

In [11]:
dataset=TensorDataset(torch.tensor(inputs,dtype=torch.float32),torch.tensor(labels,dtype=torch.float32))

In [12]:
def split_indices(n,val_pct):
    n_val=int(val_pct*n)
    idxs=np.random.permutation(n)
    return idxs[n_val:],idxs[:n_val]

In [13]:
train_indices,val_indices=split_indices(len(dataset),val_pct=0.2)
print(len(train_indices),len(val_indices))
train_indices

713 178


array([546, 200, 302, 730, 637, 274,  38, 781, 867, 789, 112, 711, 808,
       849, 349, 458, 705, 488, 768, 868, 765, 211, 341, 678, 267, 105,
       487, 180, 860,  10, 184, 644, 590, 571, 376,  19, 775, 292, 650,
        79, 305,  60, 504, 715, 281, 540,  91,  42, 641, 707, 206, 876,
       379, 703, 175, 495, 837, 842, 471, 829, 235, 364, 130, 293, 562,
       129, 215, 315, 636,  98, 476,  25, 815,  34, 671, 509, 634, 708,
       750, 621, 462, 710, 394, 685, 626, 615, 886, 245, 525, 841, 493,
       290, 126, 889, 799,   6, 872, 218, 778, 642, 519, 363, 386,  80,
       455,  82, 446, 345, 210, 844, 160, 167, 390, 415, 335, 803, 712,
       759,  99, 359, 110, 128, 237, 277, 418,  68, 132, 782, 683, 545,
       753, 555, 747, 197, 822, 714, 391, 207, 603, 843, 629,  48, 609,
       146, 858,  41, 380, 877, 763,  44, 452, 755, 786, 871, 239, 474,
       486, 256, 625, 722, 294,  11, 836, 153, 501, 716, 420, 350, 150,
       732, 611, 381, 248, 550, 196, 741, 172, 231, 820, 814, 53

In [14]:
val_indices[:20]

array([226, 317, 643, 312, 405, 631, 514, 522, 568,  15, 558, 182, 147,
       164, 821, 839, 269,  69, 209, 869])

In [15]:
batch_size=100
train_sampler=SubsetRandomSampler(train_indices)
train_dl=DataLoader(dataset,batch_size,sampler=train_sampler)
val_sampler=SubsetRandomSampler(val_indices)
val_dl=DataLoader(dataset,batch_size,sampler=val_sampler)

In [16]:
for images,labels in train_dl:
    print(images.shape)

torch.Size([100, 7])
torch.Size([100, 7])
torch.Size([100, 7])
torch.Size([100, 7])
torch.Size([100, 7])
torch.Size([100, 7])
torch.Size([100, 7])
torch.Size([13, 7])


In [42]:
for batch in train_dl:
    x,y=batch
    y=torch.reshape(y,(-1,))
    
    print(y.shape)

torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([100])
torch.Size([13])


In [41]:
input_size=7
hidden_size=5
num_classes=4
lr=0.5

In [19]:
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [43]:
class titanic(nn.Module):
    def __init__(self,input_size,hidden_size,num_classes) -> None:
        super().__init__()
        self.linear1=nn.Linear(input_size,hidden_size)
        self.linear2=nn.Linear(hidden_size,num_classes)
    def forward(self,xb):
        out=self.linear1(xb)
        out=F.relu(out)
        out=self.linear2(out)
        return out
    def training_step(self,batch):
        input,label=batch
        out=self(input)
        label=torch.reshape(label,(-1,))
        label=label.long()
        loss=F.cross_entropy(out,label)
        return loss
    def validation_step(self, batch):
        images, label = batch 
        out = self(images)
        label=torch.reshape(label,(-1,))
        label=label.long()                    # Generate predictions
        loss = F.cross_entropy(out, label)   # Calculate loss
        acc = accuracy(out, label)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    def epoch_end(self, epoch, result):
        print("Epoch [{}], val_loss: {:.4f}, val_acc: {:.4f}".format(epoch, result['val_loss'], result['val_acc']))

model=titanic(input_size,hidden_size=5,num_classes=num_classes)

In [44]:
def evaluate(model, val_loader):
    """Evaluate the model's performance on the validation set"""
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [45]:
def fit(epochs,lr,model,train_dl,val_dl,opt=torch.optim.SGD):
    history=[]
    optimizer=opt(model.parameters(),lr)
    for e in range(epochs):
        for batch in train_dl:
            loss=model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_dl)
        model.epoch_end(e+1, result)
        history.append(result)
    return history
            

In [46]:
for t in model.parameters():
    print(t.shape)

torch.Size([5, 7])
torch.Size([5])
torch.Size([4, 5])
torch.Size([4])


In [50]:
for images, labels in train_dl:
    outputs = model(images)
    labels=torch.reshape(labels,(-1,))
    labels=labels.long()
    loss = F.cross_entropy(outputs, labels)
    print('Loss:', loss.item())
    break

print('outputs.shape : ', outputs.shape)
print('Sample outputs :\n',F.softmax(outputs)[:2].data)

Loss: 6.638700008392334
outputs.shape :  torch.Size([100, 4])
Sample outputs :
 tensor([[2.1252e-02, 9.7675e-01, 1.9711e-03, 2.3855e-05],
        [2.5048e-02, 9.6779e-01, 6.9837e-03, 1.7548e-04]])


C:\Users\aabir\AppData\Local\Temp\ipykernel_24168\3130503044.py:10: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  print('Sample outputs :\n',F.softmax(outputs)[:2].data)


In [51]:
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

In [52]:
device=get_device()
device

device(type='cpu')

In [53]:
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [54]:
for images, labels in train_dl:
    print(images.shape)
    images = to_device(images, device)
    print(images.device)
    break

torch.Size([100, 7])
cpu


In [55]:
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

In [56]:
train_dl=DeviceDataLoader(train_dl,device)
val_dl=DeviceDataLoader(val_dl,device)

In [57]:
def evaluate(model, val_loader):
    """Evaluate the model's performance on the validation set"""
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [58]:
def fit(epochs,lr,model,train_dl,val_dl,opt=torch.optim.SGD):
    history=[]
    optimizer=opt(model.parameters(),lr)
    for e in range(epochs):
        for batch in train_dl:
            loss=model.training_step(batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        result = evaluate(model, val_dl)
        model.epoch_end(e+1, result)
        history.append(result)
    return history
            

In [60]:
to_device(model,device)

titanic(
  (linear1): Linear(in_features=7, out_features=5, bias=True)
  (linear2): Linear(in_features=5, out_features=4, bias=True)
)

In [59]:
evaluate(model,val_dl)

{'val_loss': 7.308977127075195, 'val_acc': 0.06846153736114502}

In [61]:
history = fit(5,0.5,model,train_dl,val_dl)

Epoch [1], val_loss: 0.8964, val_acc: 0.7247
Epoch [2], val_loss: 0.8347, val_acc: 0.7276
Epoch [3], val_loss: 0.8021, val_acc: 0.7290
Epoch [4], val_loss: 0.8012, val_acc: 0.7233
Epoch [5], val_loss: 0.7814, val_acc: 0.7276


In [62]:
history = fit(10000,5e-7,model,train_dl,val_dl)

Epoch [1], val_loss: 0.7794, val_acc: 0.7290
Epoch [2], val_loss: 0.7783, val_acc: 0.7304
Epoch [3], val_loss: 0.7640, val_acc: 0.7388
Epoch [4], val_loss: 0.7744, val_acc: 0.7332
Epoch [5], val_loss: 0.7727, val_acc: 0.7332
Epoch [6], val_loss: 0.7744, val_acc: 0.7332
Epoch [7], val_loss: 0.7833, val_acc: 0.7262
Epoch [8], val_loss: 0.7715, val_acc: 0.7346
Epoch [9], val_loss: 0.7842, val_acc: 0.7262
Epoch [10], val_loss: 0.7758, val_acc: 0.7304
Epoch [11], val_loss: 0.7830, val_acc: 0.7276
Epoch [12], val_loss: 0.7718, val_acc: 0.7332
Epoch [13], val_loss: 0.7715, val_acc: 0.7346
Epoch [14], val_loss: 0.7805, val_acc: 0.7276
Epoch [15], val_loss: 0.7729, val_acc: 0.7318
Epoch [16], val_loss: 0.7659, val_acc: 0.7374
Epoch [17], val_loss: 0.7791, val_acc: 0.7304
Epoch [18], val_loss: 0.7791, val_acc: 0.7304
Epoch [19], val_loss: 0.7715, val_acc: 0.7346
Epoch [20], val_loss: 0.7772, val_acc: 0.7318
Epoch [21], val_loss: 0.7822, val_acc: 0.7276
Epoch [22], val_loss: 0.7811, val_acc: 0.72

In [40]:
history = fit(1000,lr,model,train_dl,val_dl)

Epoch [1], val_loss: 0.9217, val_acc: 0.7290
Epoch [2], val_loss: 0.9130, val_acc: 0.7346
Epoch [3], val_loss: 0.9222, val_acc: 0.7304
Epoch [4], val_loss: 0.9116, val_acc: 0.7346
Epoch [5], val_loss: 0.9169, val_acc: 0.7304
Epoch [6], val_loss: 0.9093, val_acc: 0.7388
Epoch [7], val_loss: 0.9179, val_acc: 0.7318
Epoch [8], val_loss: 0.9328, val_acc: 0.7304
Epoch [9], val_loss: 0.9245, val_acc: 0.7276
Epoch [10], val_loss: 0.9126, val_acc: 0.7332
Epoch [11], val_loss: 0.9117, val_acc: 0.7290
Epoch [12], val_loss: 0.9172, val_acc: 0.7247
Epoch [13], val_loss: 0.9281, val_acc: 0.7247
Epoch [14], val_loss: 0.9048, val_acc: 0.7346
Epoch [15], val_loss: 0.9148, val_acc: 0.7304
Epoch [16], val_loss: 0.9178, val_acc: 0.7304
Epoch [17], val_loss: 0.9209, val_acc: 0.7276
Epoch [18], val_loss: 0.9102, val_acc: 0.7346
Epoch [19], val_loss: 0.9050, val_acc: 0.7332
Epoch [20], val_loss: 0.9089, val_acc: 0.7318
Epoch [21], val_loss: 0.9158, val_acc: 0.7262
Epoch [22], val_loss: 0.8975, val_acc: 0.73

In [65]:
submission=model.forward(test)

TypeError: linear(): argument 'input' (position 1) must be Tensor, not DataFrame

In [ ]:
df=pd.DataFrame({"PassengerId":test_ids.values,"Survived":submission})
df.to_csv("submit2.csv",index=False)